## ESTUDO DE ÁGUA MINERAL COMERCIALIZADAS EM BRASÍLIA-DF ATRAVÉS DE UM MODELO DE PCA 


O objetivo desse trabalho é, com as informações fornecidas nos próprios rótulos das marcas, tentar identificar padrões e ver como essas diferentes marcas se relacionam. Também é verificar quais marcas se relacionam e como as fontes das águas se relacionam umas com as outras. Verificar quais propriedades são mais importantes para a correlacionar as diferentes águas. Há também um dado de uma água potável, não relacionada com as águas minerais, e o objetivo com essa amostra é ver quão relacionada ela é com as outras amostras de água mineral.

In [ ]:
#Importando as Bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
#Carregando e mostrando os dados no formato dataframe
df = pd.read_csv('/mnt/c/Users/camil/OneDrive/Área de Trabalho/Scripts-python/Coursera_python_labnotes/Dados àgua Mineral Mercado_12_04_2022.csv',sep=';')
df.head()

## Tratamento dos Dados
Depois de importar as bibliotecas e o arquivo com os dados, o primeiro passo é o tratamento de dados.
Os valores NaN (nulos) das colunas de cloro, sódio, potássio e nitrato, foram substituidos pelos valores das médias da colunas. 

In [ ]:
# Substituindo os valores NaN pela média da coluna
avg_clor = df["Cloreto (mg/L)"].astype("float").mean(axis=0)
avg_sodio = df["Sódio (mg/L)"].astype("float").mean(axis=0)
avg_potassio = df["Potássio (mg/L)"].astype("float").mean(axis=0)
avg_nitrato = df["Nitrato (mg/L)"].astype("float").mean(axis=0)


print('Média Cloreto:', avg_clor)
print('Média Sódio:', avg_sodio)
print('Média Potássio:', avg_potassio)
print('Média Nitrato:', avg_nitrato)

In [ ]:
# Substituindo os valores 
df["Cloreto (mg/L)"].replace(np.nan, avg_clor, inplace=True)

In [ ]:
# Substituindo os valores 
df["Sódio (mg/L)"].replace(np.nan, avg_sodio, inplace=True)
df["Potássio (mg/L)"].replace(np.nan, avg_potassio, inplace=True)
df["Nitrato (mg/L)"].replace(np.nan, avg_nitrato, inplace=True)

Há muitas variáveis e nem todas as marcas possuem todas as informações. As variáveis escolhidas foram aquelas no qual a maioria das marcas tinham informações.

In [ ]:
# Escolhendo os dados que serão utilizados

agua_mineral_df = df[['Amostra',
                      'pH 25°C',
                      'Temperatura (C°)',
                      'Condutividade a25°C (μS/cm)',
                      'Resíduo Evaporação (mg/L)',
                      'Bicarbonato (mg/L)',
                      'Cálcio (mg/L)',
                      'Magnésio (mg/L)',
                      'Cloreto (mg/L)',
                      'Nitrato (mg/L)',
                      'Sódio (mg/L)',
                      'Potássio (mg/L)',
                      'Sulfato (mg/L)']]
agua_mineral_df

In [ ]:
agua_mineral_df.shape

In [ ]:
#Retirando a coluna "Amostras" 
agua_df = agua_mineral_df.iloc[:,1:]
agua_df

In [ ]:
agua_df.shape

## Autoescalamento dos dados 

Com a matriz de dados já preparada para ser analisada, vamos agora autoescalar os dados. Os dados serão autoescalado centrando na média.

In [ ]:
# Transformando os dados em matrizes com numpy
# Criando a matriz x com as 12 colunas de variáveis


x = agua_df.values

# Verificando o tamanho das matrizes
print(x.shape)


In [ ]:
# Mostrando a Matriz de dados x
x

In [ ]:
# Centrando os dados na média
x_mean = x-x.mean(axis=0)
x_mean

In [ ]:
# Fazendo a transposta de x para calcular o desvio padrão
transp_x = x.T
transp_x

In [ ]:
# Cálculo do desvio padrão de cada coluna
x_std = []
for column in transp_x:
  x_std.append(column.std(ddof=1))

x_std

In [ ]:
# Divisão dos dados centrados na média pelo desvio padrão

x_scaled =[]
for row in x_mean:
  x_scaled_row=[]
  for i in range(0,len(row)):
    x_scaled_row.append(row[i]/x_std[i])
  x_scaled.append(x_scaled_row)

x_scaled

In [ ]:
# Transformando em matriz a lista criada dos desvio padrão
x_scaled = np.array(x_scaled)

In [ ]:
# Verificando o novo desvio padrão para ver se os dados estão centrados na média
x_scaled.std(axis=0, ddof=1)

Apesar que alguns pacotes já vem com o autoescalamento automático como no caso do pacote "pca" que vai ser usado mais a frente. Também há outras funções que fazem o autoescalamento como o fit e o tranfsform. Nesse caso, foi feito passo a passo para diminuir o entroncamento.

## Matriz de correlações e mapa de calor

In [ ]:
# Criar nova dataframe com x_scaled data
new_df = pd.DataFrame(data = x_scaled,
                        columns=['pH 25°C',
                      'Temperatura (C°)',
                      'Condutividade a25°C (μS/cm)',
                      'Resíduo Evaporação (mg/L)',
                      'Bicarbonato (mg/L)',
                      'Cálcio (mg/L)',
                      'Magnésio (mg/L)',
                      'Cloreto (mg/L)',
                      'Nitrato (mg/L)',
                      'Sódio (mg/L)',
                      'Potássio (mg/L)',
                      'Sulfato (mg/L)'],
                      index=agua_df.index)
new_df

In [ ]:
# Cálculo da matriz de correlações 
corr = new_df.corr()
corr

In [ ]:
# Mapa de calor das correlações 
sns.heatmap(corr,
        xticklabels=corr.columns,
        yticklabels=corr.columns,cmap='YlGnBu')

Com o mapa de calor, pode-se ver que o resíduo de evaporação tem uma forte correlação com a condutividade. O cloreto também tem uma boa correlação entre o sulfato e a temperatura não tem quase correlação com o magnésio. Assim, com o mapa de calor, pode-se ver fatores que são dependentes ou independentes entre si dentro da análise de água. 

## Análise de todos os componentes

In [ ]:
from sklearn.decomposition import PCA

pcaT = PCA(n_components=12)
pcaT

In [ ]:
pcaT.fit(x_scaled)

In [ ]:
# Variância das compoentes principais
# O quanto cada componente explica 
var_ratioT = pcaT.explained_variance_ratio_

#arrendondando os valores
np.round(var_ratioT,2)

In [ ]:
# Variância Acumulada
var_ratioT.cumsum()

## Análise de 2 componentes principais

In [ ]:
# Considerando apenas 2 componentes principais
pca1=PCA(n_components=2)
pca_scores = pca1.fit(x_scaled)
pca_x = pca_scores.transform(x_scaled)

In [ ]:
pca_x.shape

In [ ]:
# Variância das compoentes principais
var_ratio = pca1.explained_variance_ratio_

#arrendondando os valores
np.round(var_ratio,2)

In [ ]:
# Melhorar a visualização
PC_escores_df = pd.DataFrame(data=pca_x,columns=['PC1', 'PC2'], index=agua_mineral_df['Amostra'])
PC_escores_df

In [ ]:
# Gráfico de  Escores PCA
fig, ax = plt.subplots(1,1,figsize=(10,5))

#plotar gráfico de pontos
ax.scatter(PC_escores_df['PC1'], PC_escores_df['PC2'])

#Adicionando legendas
for index, series in PC_escores_df.iterrows():
    ax.text(series[0]+0.05, series[1]+0.05,index)

# Ajuste gráfico
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_title("PCA - Escores (2 componentes)")
#ax.set_ylim(-3,3)
#ax.set_xlim(-2,4)
#ax.grid()

In [ ]:
#Calculando os pesos
pesos = pca1.components_
pesos.shape

In [ ]:
pesos

In [ ]:
pesosT = pesos.transpose()
pesosT

In [ ]:
#Criando um dataframe dos pesos para melhorar visualização
PC_pesos_df = pd.DataFrame(data=pesosT,
                            columns=['PC1', 'PC2'],
                            index=['pH 25°C',
                                  'Temperatura (C°)',
                                  'Condutividade a25°C (μS/cm)',
                                  'Resíduo Evaporação (mg/L)',
                                  'Bicarbonato (mg/L)',
                                  'Cálcio (mg/L)',
                                  'Magnésio (mg/L)',
                                  'Cloreto (mg/L)',
                                  'Nitrato (mg/L)',
                                  'Sódio (mg/L)',   
                                  'Potássio (mg/L)',
                                  'Sulfato (mg/L)'])
PC_pesos_df

In [ ]:
# Gráfico de pesos de PCA
fig, ax = plt.subplots(1,1,figsize=(10,5))

#plotar gráfico de pontos
ax.scatter(PC_pesos_df['PC1'], PC_pesos_df['PC2'])

#Adicionando legendas
for index, series in PC_pesos_df.iterrows():
    ax.text(series[0]+0.02, series[1]+0.02,index)

# Ajuste gráfico
ax.set_xlabel('PC 1')
ax.set_ylabel('PC 2')
ax.set_title("PCA - Pesos (2 componentes)")
#ax.set_ylim(-2,2)
#ax.set_xlim(-1,1)
#ax.grid()

# T² Hotelling
Cálculo do limite de T² Hotelling e do p-value.

In [ ]:
from sklearn import decomposition
import scipy

#Calculo do limite da elipse com os escores
theta = np.concatenate((np.linspace(-np.pi, np.pi, 50), np.linspace(np.pi, -np.pi, 50)))
circle = np.array((np.cos(theta), np.sin(theta)))
sigma = np.cov(np.array((pca_x[:, 0], pca_x[:, 1])))
ed = np.sqrt(scipy.stats.chi2.ppf(0.95, 2))
ell = np.transpose(circle).dot(np.linalg.cholesky(sigma) * ed)
a, b = np.max(ell[: ,0]), np.max(ell[: ,1]) #95% limite de confiança para  a elipse
t = np.linspace(0, 2 * np.pi, 100)



#Código adaptado de um código em R

In [ ]:
import matplotlib.cm as cm

# Gráfico de Elipse
fig, ax = plt.subplots(1,1,figsize=(12,6))


#plotar gráfico de pontos
#Adicionando cor aos pontos
colors = np.array(["red","green","blue","lime","orangered","black","orange","purple","indigo","brown","darkgray","cyan","magenta","darkgreen","hotpink"])
ax.scatter(pca_x[:, 0], pca_x[:, 1],marker='x',c=colors)


#ellipse
ax.plot(a * np.cos(t), b * np.sin(t), color = 'red')

#Adicionando legendas
for index, series in PC_escores_df.iterrows():
    ax.text(series[0]+0.05, series[1]+0.05,index)

# Ajuste gráfico
ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_title("Elipse com T² Hotelling com intervalo de 95% de confiança ")
#ax.set_ylim(-3,3)
ax.set_xlim(-3.5,6)
#ax.grid(color = 'lightgray', linestyle = '--')

Os dados que ultrapassam as bordas da elipse estão fora dos limites de confiança de 95%. 

In [ ]:
import numpy as np
from sklearn import datasets
from scipy.stats import f

def TwoSampleT2Test(X, Y):
    nx, p = X.shape
    ny, _ = Y.shape
    delta = np.mean(X, axis=0) - np.mean(Y, axis=0)
    Sx = np.cov(X, rowvar=False)
    Sy = np.cov(Y, rowvar=False)
    S_pooled = ((nx-1)*Sx + (ny-1)*Sy)/(nx+ny-2)
    t_squared = (nx*ny)/(nx+ny) * np.matmul(np.matmul(delta.transpose(), np.linalg.inv(S_pooled)), delta)
    statistic = t_squared * (nx+ny-p-1)/(p*(nx+ny-2))
    F = f(p, nx+ny-p-1)
    p_value = 1 - F.cdf(statistic)
    print(f"Test statistic: {statistic}\nDegrees of freedom: {p} and {nx+ny-p-1}\np-value: {p_value}")
    return statistic, p_value

In [ ]:
TwoSampleT2Test(pca_x, pesosT)

## Análise de PCA com o pacote pca

In [ ]:
from pca import pca
import pandas as pd
import numpy as np


Com esse pacote foi feita a mesma análise que a anterior mas utilizando o pacote pca. 

In [ ]:
#Initialize
#dentro do parenteses pode se escolher a quantidade de componentes que se quer estudar com 'n_componentes'
# Nessa parte, como não foi escolhida a quantidade de componentes, o pacote vai fazer a análise completa por definição
model = pca(normalize=True) 



#Fit transform and include the column labels and row labels
#Foi usada a matriz x antes dos valores serem autoescalados pois o autoescalamento é automático.
results = model.fit_transform(x, 
                            col_labels=['pH 25°C',
                                 'Temperatura (C°)',
                                 'Condutividade a25°C (μS/cm)',
                                  'Resíduo Evaporação (mg/L)',
                                  'Bicarbonato (mg/L)',
                                  'Cálcio (mg/L)',
                                  'Magnésio (mg/L)',
                                  'Cloreto (mg/L)',
                                  'Nitrato (mg/L)',
                                  'Sódio (mg/L)',   
                                  'Potássio (mg/L)',
                                  'Sulfato (mg/L)'],
                             row_labels=agua_mineral_df['Amostra'])


In [ ]:
# Faz a análise dos pesos 
loadings = results['loadings']
loadings

In [ ]:
#Análise dos escores
scores = results['PC']
scores

In [ ]:
# Include the outlier detection
# Gráfico de escores com a detecção de outliers representados por 'x'
model.scatter(hotellingt2=True,figsize=(12,6),legend=False,label=False)

In [ ]:
#Here again, many other options can be turned on and off
#Gráfico de pesos, escores e elipse de T² hotelling 
model.biplot(n_feat = 10,SPE=True, hotellingt2=True,figsize=(12,6),legend=False,title='Gráfico de Pesos, Escores e limite de T² Hotelling')



Outlier pode ser detectado usando SPE/DmodX (distância ao modelo) com base na média e covariância das 2 primeiras dimensões de X. No plano do modelo (SPE ≈ 0). Note-se que o SPE ou o T2 da Hotelling são complementares entre si

A extração das variáveis de melhor desempenho é baseada nos pesos dos Componentes Principais, que são prontamente calculados. As informações são armazenadas no próprio objeto e podemos extraí-las conforme mostrado abaixo.

In [ ]:
# Determinação do desempenho das variáveis e quais são mais importantes para explicar as amostras
print(model.results['topfeat'])


A maior variância é explicada pelo PC1 dado pelo resíduo de evaporação. Para o PC2, a maior variância é explicada pelo Cloreto. 
A partir da temperatura, a variância não é tão bem explicada


O gráfico a seguir a variância acumulada dos componentes principais por cada coponente principal. 
Nesse estudo, já se sabe pela seção anterior, que 5 componentes principais explicam 95% da variância e não 98,2% como mostrado nesse gráfico. O próprio cruzamento das linhas vermelhas estão erradas já que não estão em cima do ponto de componente principal 6.

In [ ]:
model.plot()

#Infelizmente esse gráfico não é confiável. Pessoalmente acredito que deve ter algum erro no código. 

Variância acumulada e variância explicada por cada componente principal. Confere com a seção anterior.

In [ ]:
# Cumulative explained variance
print(model.results['explained_var'])


# Explained variance per PC
print(model.results['variance_ratio'])


## Dendograma

O dendograma verifica a similaridade entre as amostras através da distância. 
O dendograma do plotly verifica as distâncias pela distância Euclidiana.

In [ ]:
import plotly.figure_factory as ff
import numpy as np

amostras = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']
#color_dend = ["red","green","blue","lime","orangered","black","orange","purple","indigo","brown","darkgray","cyan","magenta","darkgreen","hotpink"]
fig = ff.create_dendrogram(x_scaled,color_threshold=3.5,labels=amostras)
fig.update_layout(width=1000, height=600)
fig.show()

Foi bem possível estudar as amostras de água mineral através do modelo PCA. 
Um relatório mais detalhado sobre os resultados e a conclusão estão disponíveis para leitura.